# Topic_Modeling_On_Tweets_Around_COVID19
In this notebook, we explore the various topics people are talking about Corona Virus Disease 2019(COVID-19) in Twitter.

In [1]:
# Import all required libraries
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tweepy as tw
import json
import requests
import re

'''
Loading Gensim and nltk libraries
'''
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

import nltk
nltk.download('wordnet')

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])
# Other stop_words: gensim.parsing.preprocessing.STOPWORDS


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/deepakawari/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Configure notebook display to show data from pandas dataframe more clearly.
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)
pd.set_option('display.width',150)
pd.set_option('display.max_colwidth',1000)

# 1. Loading data
If the tweets data file is available, then load the data from the file else read the tweets from the Tweepy API. 

## Loading data from Twitter API:
To load the data from Twitter using Tweepy API, you'll have to create Developer account with Twitter. Then download the credentials to authenticate using Tweepy API. Please do not share these credentials with anybody else. 
* Here is the link to [apply for twitter developer access](https://developer.twitter.com/en/apply-for-access)
* You can follow the below code to use Tweepy API to authenticate and load the data. Here is the [Tweepy Documentation for reference](http://docs.tweepy.org/en/latest/) 

In [3]:
'''
Load the data. Set the LoadFromTwitter to True if you want to override loading the data afresh from twitter.
'''
LoadFromTwitter = False

fileName = '../data/tweets.csv'
tweetsDF = None

# Load the data
if os.path.exists(fileName) and not LoadFromTwitter:
    tweetsDF = pd.read_csv(fileName)
else:
    from TwitterDevSecrets import getTwitterDevCreds
    consumer_key, consumer_secret, access_token, access_secret = getTwitterDevCreds()

    auth = tw.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)

    # Set the wait_on_rate_limit and wait_on_rate_limit_notify to True
    # wait_on_rate_limit – 
    #    Whether or not to automatically wait for rate limits to replenish
    # wait_on_rate_limit_notify – 
    #    Whether or not to print a notification when Tweepy is waiting 
    #    for rate limits to replenish
    api = tw.API(
        auth, 
        wait_on_rate_limit=True, 
        wait_on_rate_limit_notify=True)

    # Define the search term and the date_since date as variables
    search_words = "#covid OR #covid19 OR #COVID OR #COVID19 OR #ncov OR #corona OR #coronaviru"
    date_since = "2020-03-16"
    
    # Read the tweets
    tweets = tw.Cursor(api.search, 
                   q=search_words,
                   lang="en",
                   since=date_since)

    # extract the data in pandas dataframe
    # Other parameters: tweet.user.screen_name, retweet_counts, favorite_counts
    tweetsDF = pd.DataFrame()
    for tweet in tweets.items(1000):
        id = tweet.id
        text = tweet.text
        loc = tweet.user.location
        tweetsDF = tweetsDF.append({'Id':id, 'Text':text, 'Location':loc},ignore_index=True)
    
    tweetsDF['index'] = tweetsDF.index
    
    # Save the new set of tweets in the file.
    tweetsDF.to_csv(fileName,index=False)

# Tweets loaded
tweetsDF.head()

,Id,Location,Text,index
0,1.246511e+18,NaN,RT @ALPublicHealth: State Health Officer Dr. Scott Harris has issued a stay at home order and strict quarantine requirements. Read our full…,0
1,1.246511e+18,"Portland, oregon",RT @Carol_D_Johnson: Thank you nurses for helping to keep us healthy ❤ #COVID19 \n#StayHomeSaveLives \n#coronavirus https://t.co/HGv0HfuTgt,1
2,1.246511e+18,NaN,RT @Surgeon_General: #TogetherApart we can slow the spread of #COVID19. https://t.co/8JIBxQFpjv,2
3,1.246511e+18,NaN,RT @SkyNews: Are smokers at greater risk of contracting #coronavirus?\n\nDr Ellie Cannon says while we are all at equal risk of contracting #…,3
4,1.246511e+18,NaN,RT @evankirstel: 😱 The video of a 3D model from a CT scan shows the extent to which the #COVID19 has damaged the lung tissue #StayHome #St…,4


In [4]:
tweetsDF.Text[1]

'RT @Carol_D_Johnson: Thank you nurses for helping to keep us healthy  ❤ #COVID19 \n#StayHomeSaveLives \n#coronavirus https://t.co/HGv0HfuTgt'

# 2: Data preprocessing
We will perform the following data processing steps:

* Tweet Preprocessing:
> * Remove the leading **RT** - RT indicates that the user is re-posting someone else's tweet. We can remove this token.
> * Remove the references to other accounts. The other accounts are usually referenced with '@' symbol.
> * Remove urls mentioned in the tweets.

* Generic text preprocessing:
> * **Tokenization**: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
> * Remove words that have fewer than 3 characters.
> * Remove all **stopwords**.
> * **Lemmatize** the words: words in third person are changed to first person and verbs in past and future tenses are changed into present.  
> Lemmatization, unlike Stemming, reduces the inflected words properly ensuring that the root word belongs to the language. In Lemmatization root word is called Lemma. A lemma (plural lemmas or lemmata) is the canonical form, dictionary form, or citation form of a set of words. 
>> WordnetLemmatizer: uses lookup table from nltk wordnet corpus to lookup the lemma to return a valid language lemma.
> * **Stem** the Words: words are reduced to their root form.  
> Stemming is the process of reducing inflection in words to their root forms such as 
mapping a group of words to the same stem even if the stem itself is not a valid word 
in the Language.
>> PorterStemmer: is known for simplicity and ease. The algorithm does not follow linguistics rather a set of 05 rules for different cases that are applied in phases (step by step) to generate stems. This is the reason why PorterStemmer does not often generate stems that are actual English words.
>> SnowballStemmer: One can generate its own set of rules for any language that is why Python nltk introduced SnowballStemmers that are used to create non-English Stemmers!
>> LancasterStemmer: is simple, but heavy stemming due to iterations and over-stemming may occur. Over-stemming causes the stems to be not linguistic, or they may have no meaning.
    

In [5]:
# Perform data preprocessing for all tweets.

stemmer = SnowballStemmer("english")

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def tweet_cleanup(text):
    # Remove the leading RT from the tweet
    text = text.replace('RT','')
    # Remove the references to the account names starting with '@'
    text = re.sub(r'(@[a-zA-Z]*)','',text)
    # Remove the urls in the tweet.
    text = re.sub(r'((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)','',text)
    
    return text
  
# Tokenize and lemmatize
def preprocess(text, stop_words=stop_words):
    result=[]
    text = tweet_cleanup(text)
    for token in gensim.utils.simple_preprocess(text) :
        if token not in stop_words and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result



In [6]:
# Test the preprocessing and on a sample tweet
tweet_num = 0
sampleTweet = tweetsDF[tweetsDF['index'] == tweet_num].Text.iloc[0]

print("Original tweet: ")
words = []
for word in sampleTweet.split():
    words.append(word)
print(words)
print("\n\nPreprocessed tweet: ")
print(preprocess(sampleTweet))

Original tweet: 
['RT', '@ALPublicHealth:', 'State', 'Health', 'Officer', 'Dr.', 'Scott', 'Harris', 'has', 'issued', 'a', 'stay', 'at', 'home', 'order', 'and', 'strict', 'quarantine', 'requirements.', 'Read', 'our', 'full…']


Preprocessed tweet: 
['state', 'health', 'offic', 'scott', 'harri', 'issu', 'stay', 'home', 'order', 'strict', 'quarantin', 'requir', 'read', 'full']


In [7]:
# Preprocess all tweets and generate a new processed tweet text dataset.

processed_tweets = tweetsDF['Text'].map(preprocess)
processed_tweets[:10]

0        [state, health, offic, scott, harri, issu, stay, home, order, strict, quarantin, requir, read, full]
1                                                             [nurs, help, keep, healthi, covid, coronavirus]
2                                                                        [togetherapart, slow, spread, covid]
3                    [smoker, greater, risk, contract, coronavirus, elli, cannon, say, equal, risk, contract]
4                                      [video, model, scan, show, extent, covid, damag, lung, tissu, stayhom]
5                      [leader, hous, parti, caucus, arizona, andi, bigg, think, spread, covid, much, possib]
6                                                                                     [covid, test, administ]
7           [keep, think, master, public, health, write, doctor, dissert, global, effort, tackl, aid, pandem]
8    [ceylonblacktea, rich, theaflavin, help, increas, human, immun, covid, srilankatea, industri, successfu]
9         

# 3.1: Bag of words on the dataset
Create a dictionary of words present in the preprocessed_tweets dataset. Gensim offers a great api for the same. This dictionary assigns a numerical id to each word so that you can work on the number representations of the word. This makes the data processing very easy than working on strings. 

Then create a corpus of Bag of words where words are represented by their numerical ids along with the frequency of occurence of that word in the tweet for further processing.

In [9]:
dictionary = gensim.corpora.Dictionary(processed_tweets)

# Create Corpus: Term Document Frequency
corpus = [dictionary.doc2bow(text) for text in processed_tweets]

In [10]:
# Check the id to word mapping from the dictionary created above
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 5:
        break

0 full
1 harri
2 health
3 home
4 issu
5 offic


Since the text corpus is very huge and sparse, we should try to minimize the amount of text being used for modeling. For this reason, let us remove very rare and very common words. Gensim dictionary object provides a good api to perform this operation.
- words appearing less than 15 times
- words appearing in more than 10% of all documents

Then convert it into bag of word corpus with very rare and very common wordsd filtered out.

In [11]:
dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_tweets]

# Test the Bag of Words representation of the tweet --> (token_id, token_count)
bow_corpus[tweet_num]

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]

In [13]:
# Preview BOW for our sample preprocessed tweet
bow_tweet_0 = bow_corpus[tweet_num]

for i in range(len(bow_tweet_0)):
    print("Word {} (\"{}\") appears {} time.".format(bow_tweet_0[i][0], 
                                                     dictionary[bow_tweet_0[i][0]], 
                                                     bow_tweet_0[i][1]))

Word 0 ("health") appears 1 time.
Word 1 ("home") appears 1 time.
Word 2 ("order") appears 1 time.
Word 3 ("state") appears 1 time.
Word 4 ("stay") appears 1 time.


# 3.2: TF-IDF on the data set
TFIDF, short for term frequency–inverse document frequency, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus. Summing the Tf-idf of all possible terms and documents recovers the mutual information between documents and term taking into account all the specificities of their joint distribution.

TF (Term Frequency) - number of times a term occurs in a document.
IDF (Inverse Document Frequency) diminishes the weight of terms that occur very frequently in the document set and increases the weight of terms that occur rarely.

In [14]:
# Create tf-idf model object using models.TfidfModel
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)

# Apply transformation to the entire corpus
corpus_tfidf = tfidf[bow_corpus]

In [15]:
# Preview TF-IDF for our sample preprocessed tweet
tfidf_tweet_0 = corpus_tfidf[tweet_num]

for i in range(len(tfidf_tweet_0)):
    print("Word {} (\"{}\") TF-IDF score: {}.".format(tfidf_tweet_0[i][0], 
                                                     dictionary[tfidf_tweet_0[i][0]], 
                                                     tfidf_tweet_0[i][1]))

Word 0 ("health") TF-IDF score: 0.40009295170061265.
Word 1 ("home") TF-IDF score: 0.44472924895798494.
Word 2 ("order") TF-IDF score: 0.45253501051552114.
Word 3 ("state") TF-IDF score: 0.47433139975516847.
Word 4 ("stay") TF-IDF score: 0.4608289406979316.


# Step 4.1: Topic modeling using bag of words
In the topic modeling task, we'll have to provide the number of topics we want the model to cluster the tweets into. But how do we identify the number of topics? The best way to identify that is by visualizing the clusters itself. Start with a high number of topics like 10 or 20. Then map the clusters into a vector space and see if the clusters have clear boundaries. If the clusters overlap, reduce the number of clusters and visualize again. Repeat the process until you are satisfied with the segregation of the clusters.

## Step 4.1.1: Running LDA using bag of words

In [26]:
# Train the lda model using gensim.models.LdaMulticore on Bag of word corpus
lda_model = gensim.models.LdaMulticore(bow_corpus, 
                                       num_topics=3, 
                                       id2word = dictionary, 
                                       passes = 2, 
                                       workers=2)

In [27]:
# Explore the words occuring in that topic and its relative weight
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic))
    print("\n")

Topic: 0 
Words: 0.075*"case" + 0.044*"peopl" + 0.032*"health" + 0.031*"time" + 0.028*"mask" + 0.025*"spread" + 0.025*"trump" + 0.024*"state" + 0.024*"help" + 0.022*"face"


Topic: 1 
Words: 0.064*"coronavirus" + 0.044*"work" + 0.041*"natur" + 0.039*"speak" + 0.037*"caus" + 0.036*"father" + 0.036*"murder" + 0.035*"trut" + 0.035*"poverti" + 0.035*"racism"


Topic: 2 
Words: 0.044*"home" + 0.039*"fight" + 0.036*"stay" + 0.034*"test" + 0.034*"make" + 0.029*"say" + 0.027*"never" + 0.027*"public" + 0.027*"nurs" + 0.026*"live"




From the top words in each topic, we can identify the generic topic in that cluster. In the above clustering, the topics could be around  
Topic 0: Take precautions    
Topic 1: Impact of COVID-19 on work, racism, and poverty.   
Topic 2: Quarantine and fight Corona virus  

# 4.1.2. Visualization using pyLDAVis for LDA with BOW

In [25]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary=lda_model.id2word)
vis

/usr/local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.202957  0.006357       1        1  39.603428
0      0.095035 -0.153361       2        1  33.059582
2      0.107922  0.147004       3        1  27.336996, topic_info=    Category       Freq         Term      Total  loglift  logprob
48   Default  81.000000         case  81.000000  30.0000  30.0000
39   Default  54.000000         work  54.000000  29.0000  29.0000
34   Default  44.000000        natur  44.000000  28.0000  28.0000
94   Default  27.000000         face  27.000000  27.0000  27.0000
32   Default  38.000000       father  38.000000  26.0000  26.0000
5    Default  91.000000  coronavirus  91.000000  25.0000  25.0000
14   Default  48.000000       pandem  48.000000  24.0000  24.0000
35   Default  37.000000      poverti  37.000000  23.0000  23.0000
38   Default  37.000000         trut  37.000000  22.0000  22.0000
33   Default  38.000000       murder  38.000000  21.0000  21.0000
36   Default  37.000000       racism  37.000000  20.0000  20.0000
31   Default  39.000000         caus  39.000000  19.0000  19.0000
52   Default  28.000000       affect  28.000000  18.0000  18.0000
37   Default  41.000000        speak  41.000000  17.0000  17.0000
42   Default  39.000000        death  39.000000  16.0000  16.0000
99   Default  19.000000        cover  19.000000  15.0000  15.0000
9    Default  26.000000       spread  26.000000  14.0000  14.0000
86   Default  23.000000        break  23.000000  13.0000  13.0000
21   Default  18.000000        often  18.000000  12.0000  12.0000
16   Default  25.000000        human  25.000000  11.0000  11.0000
60   Default  19.000000    discrimin  19.000000  10.0000  10.0000
93   Default  22.000000      confirm  22.000000   9.0000   9.0000
10   Default  26.000000          say  26.000000   8.0000   8.0000
65   Default  38.000000        fight  38.000000   7.0000   7.0000
22   Default  43.000000         mask  43.000000   6.0000   6.0000
78   Default  27.000000        never  27.000000   5.0000   5.0000
74   Default  20.000000         amaz  20.000000   4.0000   4.0000
91   Default  17.000000    recommend  17.000000   3.0000   3.0000
15   Default  38.000000       public  38.000000   2.0000   2.0000
75   Default  17.000000         ceas  17.000000   1.0000   1.0000
34    Topic1  43.139221        natur  44.218315   0.9015  -3.2406
35    Topic1  36.234459      poverti  37.142269   0.9015  -3.4150
32    Topic1  37.877556       father  38.927315   0.8989  -3.3707
38    Topic1  36.122597         trut  37.165123   0.8978  -3.4181
36    Topic1  36.059978       racism  37.176247   0.8958  -3.4198
33    Topic1  36.748150       murder  38.062729   0.8911  -3.4009
39    Topic1  52.983028         work  54.947563   0.8898  -3.0351
31    Topic1  38.004578         caus  39.949383   0.8763  -3.3673
25    Topic1  17.515055     lockdown  18.678251   0.8620  -4.1420
37    Topic1  38.757122        speak  41.997864   0.8460  -3.3477
83    Topic1  15.165315          die  16.984255   0.8130  -4.2860
65    Topic1  33.359013        fight  38.469715   0.7837  -3.4977
18    Topic1  16.860672        first  19.728289   0.7692  -4.1800
67    Topic1  23.774128         make  27.948059   0.7645  -3.8364
50    Topic1  12.893298       enough  15.261683   0.7576  -4.4483
69    Topic1  12.834095        great  15.315532   0.7495  -4.4529
81    Topic1  21.299414        pleas  26.352476   0.7134  -3.9464
49    Topic1  14.520037      countri  18.024952   0.7100  -4.3295
46    Topic1  23.134855         time  28.936983   0.7025  -3.8637
30    Topic1  11.540707      support  14.662800   0.6868  -4.5591
24    Topic1  16.529530        april  22.207619   0.6310  -4.1999
57    Topic1  13.431174      respons  18.447659   0.6089  -4.4075
11    Topic1  33.750023         test  47.708038   0.5801  -3.4860
26    Topic1  14.037117       nation  20.148197   0.5648  -4.3633
95    Topic1  14.757624       presid  21.274828   0.5605 

Based on the visualizations, it'd be best to create 3 clusters instead of 10.

# 4.2.1. Running LDA using TF-IDF

In [34]:
# Train lda model using corpus_tfidf
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, 
                                             num_topics=3, 
                                             id2word = dictionary, 
                                             passes = 2, 
                                             workers=4)

In [35]:
# Explore the words occuring in that topic and its relative weight
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

Topic: 0 Word: 0.053*"peopl" + 0.042*"pandem" + 0.033*"work" + 0.026*"trump" + 0.025*"state" + 0.025*"help" + 0.025*"natur" + 0.024*"send" + 0.024*"govern" + 0.024*"case"


Topic: 1 Word: 0.037*"patient" + 0.036*"death" + 0.033*"mask" + 0.031*"time" + 0.031*"case" + 0.030*"coronavirus" + 0.026*"lockdown" + 0.026*"nation" + 0.024*"ask" + 0.024*"face"


Topic: 2 Word: 0.058*"coronavirus" + 0.040*"health" + 0.036*"test" + 0.031*"spread" + 0.031*"say" + 0.030*"make" + 0.027*"fight" + 0.025*"die" + 0.024*"stay" + 0.024*"pleas"




Topics from the above words:  
Topic 0: President Trump's announcements   
Topic 1: Impact of COVID in terms of patients, deaths and lockdowns.   
Topic 2: Quarantine and fight the spread of COVID-19

# 4.2.2. Visualization using pyLDAVis

In [37]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_tfidf, corpus_tfidf, dictionary=lda_model_tfidf.id2word)
vis

/usr/local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.146376 -0.040145       1        1  35.122742
0      0.112811 -0.091155       2        1  34.296890
1      0.033564  0.131300       3        1  30.580362, topic_info=    Category       Freq         Term      Total  loglift  logprob
45   Default  18.000000      patient  18.000000  30.0000  30.0000
14   Default  24.000000       pandem  24.000000  29.0000  29.0000
42   Default  19.000000        death  19.000000  28.0000  28.0000
20   Default  35.000000        peopl  35.000000  27.0000  27.0000
0    Default  24.000000       health  24.000000  26.0000  26.0000
9    Default  17.000000       spread  17.000000  25.0000  25.0000
5    Default  46.000000  coronavirus  46.000000  24.0000  24.0000
25   Default  13.000000     lockdown  13.000000  23.0000  23.0000
26   Default  13.000000       nation  13.000000  22.0000  22.0000
94   Default  11.000000         face  11.000000  21.0000  21.0000
23   Default  13.000000         send  13.000000  20.0000  20.0000
100  Default  10.000000       releas  10.000000  19.0000  19.0000
10   Default  19.000000          say  19.000000  18.0000  18.0000
84   Default  12.000000          ask  12.000000  17.0000  17.0000
72   Default  12.000000        equip  12.000000  16.0000  16.0000
73   Default  11.000000         look  11.000000  15.0000  15.0000
11   Default  24.000000         test  24.000000  14.0000  14.0000
83   Default  14.000000          die  14.000000  13.0000  13.0000
24   Default  13.000000        april  13.000000  12.0000  12.0000
46   Default  19.000000         time  19.000000  11.0000  11.0000
99   Default   8.000000        cover   8.000000  10.0000  10.0000
33   Default  13.000000       murder  13.000000   9.0000   9.0000
70   Default  13.000000       worker  13.000000   8.0000   8.0000
47   Default  11.000000       import  11.000000   7.0000   7.0000
12   Default  12.000000       doctor  12.000000   6.0000   6.0000
8    Default  12.000000         nurs  12.000000   5.0000   5.0000
56   Default  11.000000         must  11.000000   4.0000   4.0000
38   Default  13.000000         trut  13.000000   3.0000   3.0000
67   Default  19.000000         make  19.000000   2.0000   2.0000
3    Default  16.000000        state  16.000000   1.0000   1.0000
73    Topic1  10.782625         look  11.817240   0.9547  -3.8485
60    Topic1   7.304064    discrimin   8.139240   0.9381  -4.2381
41    Topic1   9.102310         isol  10.175518   0.9349  -4.0180
9     Topic1  15.854310       spread  17.992069   0.9198  -3.4630
56    Topic1  10.368087         must  11.774825   0.9191  -3.8878
58    Topic1   5.227725         basi   5.938187   0.9189  -4.5725
59    Topic1   5.224496         cast   5.937832   0.9183  -4.5731
64    Topic1   5.211537     religion   5.937471   0.9159  -4.5756
17    Topic1   6.008577         rich   6.932136   0.9033  -4.4333
16    Topic1  10.698136        human  12.398549   0.8988  -3.8564
83    Topic1  12.476450          die  14.510469   0.8953  -3.7026
0     Topic1  20.436157       health  24.918278   0.8480  -3.2092
80    Topic1  10.306734         week  12.641532   0.8421  -3.8937
63    Topic1   7.364781         poor   9.200249   0.8238  -4.2298
10    Topic1  15.484200          say  19.415380   0.8201  -3.4867
67    Topic1  15.122181         make  19.639425   0.7849  -3.5103
61    Topic1   5.931641       either   7.762289   0.7773  -4.4462
11    Topic1  18.353014         test  24.958193   0.7389  -3.3167
21    Topic1   4.992930        often   6.856561   0.7291  -4.6185
43    Topic1  10.785190       report  14.912615   0.7223  -3.8483
69    Topic1   7.661205        great  10.889621   0.6947  -4.1903
81    Topic1  12.313420        pleas  17.540483   0.6925  -3.7158
4     Topic1  12.357800         stay  17.648344   0.6900  -3.7122
76    Topic1   4.302457      gullibl   6.218040   0.6781  -4.7673
75    Topic1   4.270602         ceas   6.215453   0.6710 

May be 3 topics from the visualizations. 